In [1]:
import os, glob, sys
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Polygon, mapping
import matplotlib.pyplot as plt
import folium
import rasterio
import rasterio.mask
import rasterio.plot
import numpy as np

sys.path.insert(0, '/export/miro/ndeffense/LBRAT2104/GIT/eo-toolbox/tools/')

import folium_def


# Input / Output

## Input

In [2]:
path = '/export/projects/FAO-EOStat-Senegal/'

odk_crop_csv     = f'{path}ODK/FormSenegal05.csv'
odk_non_crop_csv = f'{path}ODK/FormSenegal07.csv'

odk_shp = f'{path}ODK/polygons_ODK.shp'
roi_shp = f'{path}ROI/Senegal_DepartementNioro.shp'

lut_crop_csv     = f'{path}LUT/LUT_crop.csv'
lut_non_crop_csv = f'{path}LUT/LUT_non_crop.csv'

## Output

In [20]:
gps_shp_output          = f'{path}Output_Nico/gps_with_data_with_buffer.shp'
odk_crop_shp_output     = f'{path}Output_Nico/odk_crop_with_data.shp'
odk_non_crop_shp_output = f'{path}Output_Nico/odk_non_crop_with_data.shp'

full_shp_output = f'{path}Sen4Stat/IN_SITU/SITE_41/SEN_2021_SITE_41_ori.shp'

map_html_output = f'{path}Output_Nico/odk_gps_with_data.html'


# Parameters

In [4]:
buf_size = -10  # meter
min_area = 100  # square meter

# Open Look-up-Table

In [5]:
lut_crop_df = pd.read_csv(lut_crop_csv)

lut_crop_df.head()

,crop_code,crop_name
0,1,Arachide
1,2,Aubergine
2,3,Béréf
3,4,Bissap
4,5,Coton


In [6]:
lut_non_crop_df = pd.read_csv(lut_non_crop_csv)

lut_non_crop_df.head()

,lc_code,lc_name
0,1,Prairie
1,2,Jachère
2,3,Broussailes
3,4,Forêt
4,5,Sol nu


# GPX files

In [7]:
dict_gpx_list = []

wrong_pid = 0

for gpx_file in glob.glob(f'{path}GPS/*/*.gpx'):

    filename = os.path.basename(gpx_file)

    
    pid = filename[filename.find("Piste")+5:-4].lstrip('_ ')
    pid = pid.replace(' ','-')
    pid = pid.replace('---','-')
    pid = pid.replace('--','-')
    pid = pid.split('-')

    
    if len(pid) <= 5:
        pid = ''.join(pid)

        layer = fiona.open(gpx_file, layer='tracks')

        crs = layer.crs

        if crs['init'] == 'epsg:4326':

            dict_gpx = {'filename': filename,
                        'id': pid,
                        'coordinates': layer[0]['geometry']['coordinates'][0]}

            dict_gpx_list.append(dict_gpx)
    else:
        wrong_pid += 1

print(f'Good PID : {len(dict_gpx_list)}')
print(f'Wrong PID : {wrong_pid}')


Good PID : 838
Wrong PID : 36


## Store GPX tracks in GeoDataFrame

- Apply "zero" buffer to clean geometries
- Drop duplicates

In [28]:
df = pd.DataFrame.from_dict(dict_gpx_list)

df['geometry'] = df.coordinates.apply(Polygon)

df = df.drop('coordinates', axis=1)

gps_gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry').to_crs(epsg=32628)

gps_gdf = gps_gdf.drop_duplicates()

gps_gdf['geometry'] = gps_gdf.buffer(buf_size)

gps_gdf['area'] = gps_gdf['geometry'].area.round(2)

gps_gdf = gps_gdf.loc[gps_gdf['area'] >= min_area]

gps_gdf['collect'] = 'GPS'


print(f'There are {len(gps_gdf)} polygons obtained with the GPS')
gps_gdf.head()



There are 321 polygons obtained with the GPS


,filename,id,geometry,area,collect
0,Piste_6220102-2-4-1-1.gpx,62201022411,"POLYGON ((429457.856 1516229.002, 429465.169 1...",5172.41,GPS
1,Piste_6220102-2-4-1-2.gpx,62201022412,"POLYGON ((429828.180 1516150.340, 429839.170 1...",12162.62,GPS
2,Piste_6220102-2-4-1-3.gpx,62201022413,"POLYGON ((429883.899 1516457.550, 429884.205 1...",6209.42,GPS
3,Piste_6220102-2-4-1-4.gpx,62201022414,"POLYGON ((429986.736 1516281.999, 429987.433 1...",2772.11,GPS
4,Piste_6220102-2-16-4.gpx,62201022164,"POLYGON ((429722.291 1515343.246, 429740.288 1...",1860.82,GPS


In [27]:
gdf = gpd.overlay(gps_gdf, gps_gdf, how='intersection')

gdf


,filename_1,id_1,area_1,collect_1,filename_2,id_2,area_2,collect_2,geometry
0,Piste_6220102-2-4-1-1.gpx,62201022411,5172.41,GPS,Piste_6220102-2-4-1-1.gpx,62201022411,5172.41,GPS,"POLYGON ((429457.856 1516229.002, 429465.169 1..."
1,Piste_6220102-2-4-1-2.gpx,62201022412,12162.62,GPS,Piste_6220102-2-4-1-2.gpx,62201022412,12162.62,GPS,"POLYGON ((429828.180 1516150.340, 429839.170 1..."
2,Piste_6220102-2-4-1-3.gpx,62201022413,6209.42,GPS,Piste_6220102-2-4-1-3.gpx,62201022413,6209.42,GPS,"POLYGON ((429883.899 1516457.550, 429884.205 1..."
3,Piste_6220102-2-4-1-4.gpx,62201022414,2772.11,GPS,Piste_6220102-2-4-1-4.gpx,62201022414,2772.11,GPS,"POLYGON ((429986.736 1516281.999, 429987.433 1..."
4,Piste_6220102-2-16-4.gpx,62201022164,1860.82,GPS,Piste_6220102-2-16-4.gpx,62201022164,1860.82,GPS,"POLYGON ((429722.291 1515343.246, 429740.288 1..."
...,...,...,...,...,...,...,...,...,...
324,Piste_6220206-7-7-2-2.gpx,62202067722,4393.56,GPS,Piste_6220206-7-7-2-2.gpx,62202067722,4393.56,GPS,"POLYGON ((423666.487 1531218.677, 423664.305 1..."
325,Piste_6220206-7-7-2-3.gpx,62202067723,6802.07,GPS,Piste_6220206-7-7-2-3.gpx,62202067723,6802.07,GPS,"POLYGON ((425034.700 1531408.852, 425036.310 1..."
326,Piste_6220206-7-7-2-4.gpx,62202067724,6438.62,GPS,Piste_6220206-7-7-2-4.gpx,62202067724,6438.62,GPS,"POLYGON ((425355.728 1531218.177, 425355.744 1..."
327,Piste_6220206-7-7-2-5.gpx,62202067725,1216.52,GPS,Piste_6220206-7-7-2-5.gpx,62202067725,1216.52,GPS,"POLYGON ((425303.273 1531277.982, 425304.945 1..."


# ODK

## Load cropland data from ODK

- Create ID based on fields

In [10]:
odk_crop_df = pd.read_csv(odk_crop_csv)

odk_crop_df['id'] = odk_crop_df['id-Commune'].astype(str) + odk_crop_df['id-District'].astype(str) + odk_crop_df['id-Concession'].astype(str) + odk_crop_df['id-Menage'].astype(str) + odk_crop_df['id-Parcelle'].astype(str)

odk_crop_df = odk_crop_df[['id','Informations_champ-Melange','Informations_champ-Culture','Informations_champ-Surface']]

odk_crop_df['Informations_champ-Surface'] = odk_crop_df['Informations_champ-Surface'].round(2)


odk_crop_df = odk_crop_df.merge(lut_crop_df, left_on='Informations_champ-Culture', right_on='crop_code')


odk_crop_df = odk_crop_df.drop_duplicates(subset=['id'])

print(f'There are {len(odk_crop_df)} rows in the ODK csv.')

odk_crop_df



There are 231 rows in the ODK csv.


,id,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name
0,6220201174341,1,1.0,31297.99,1,Arachide
1,6220201171611,1,1.0,17649.58,1,Arachide
2,6220201171631,1,1.0,10117.53,1,Arachide
3,6220201171661,1,1.0,7216.51,1,Arachide
4,6220201173221,1,1.0,7180.45,1,Arachide
...,...,...,...,...,...,...
230,62201022539103,1,9.0,8774.09,9,Maïs
231,6220205512446,1,9.0,12192.00,9,Maïs
232,6220303428444,1,9.0,78.58,9,Maïs
233,6220102321131,1,8.0,358.62,8,Gombo


## Load cropland polygons from ODK

- Convert Linstring to Polygon
- Drop duplicates
- Apply "zero" buffer to clean geometries

In [11]:
odk_gdf = gpd.read_file(odk_shp)

odk_gdf['geometry'] = [Polygon(mapping(x)['coordinates']) for x in odk_gdf.geometry]

odk_gdf['id'] = odk_gdf['id-Commune'].astype(str) + odk_gdf['id-Distric'].astype(str) + odk_gdf['id-Concess'].astype(str) + odk_gdf['id-Menage'].astype(str) + odk_gdf['id-Parcell'].astype(str)

#odk_gdf = odk_gdf[['id','Informations_champ-Melange','Informations_champ-Culture','Informations_champ-Surface','geometry']]

odk_gdf = odk_gdf.to_crs(epsg=32628)

odk_gdf = odk_gdf[['id','geometry']]

odk_gdf = odk_gdf.drop_duplicates(subset=['id'])


odk_gdf['geometry'] = odk_gdf.buffer(buf_size)

odk_gdf['area'] = odk_gdf['geometry'].area.round(2)

odk_gdf = odk_gdf.loc[odk_gdf['area'] >= min_area]


odk_gdf['collect'] = 'ODK_polygon'


print(f'There are {len(odk_gdf)} polygons obtained with the tablet (ODK)')

odk_gdf.head()

There are 215 polygons obtained with the tablet (ODK)


,id,geometry,area,collect
0,6220201174341,"POLYGON ((393917.866 1528882.489, 393901.785 1...",24112.55,ODK_polygon
1,6220201173222,"POLYGON ((394098.110 1528308.033, 394129.216 1...",4709.83,ODK_polygon
2,6220201171611,"POLYGON ((394069.867 1529538.943, 394187.774 1...",12635.46,ODK_polygon
3,6220201171631,"POLYGON ((393943.188 1529558.847, 393938.597 1...",6242.19,ODK_polygon
4,6220201171661,"POLYGON ((393939.067 1529497.632, 393938.507 1...",3883.07,ODK_polygon


## Load non-cropland data from ODK

In [12]:
odk_non_crop_df = pd.read_csv(odk_non_crop_csv)

odk_non_crop_df = odk_non_crop_df[['Picture-LC_Class','Picture-Other_Non_Cropland','start-geopoint-Latitude','start-geopoint-Longitude']]

odk_non_crop_df = odk_non_crop_df.rename(columns={"start-geopoint-Longitude": "longitude", "start-geopoint-Latitude": "latitude"})

odk_non_crop_df = odk_non_crop_df.merge(lut_non_crop_df, left_on='Picture-LC_Class', right_on='lc_code')

odk_non_crop_gdf = gpd.GeoDataFrame(odk_non_crop_df, geometry=gpd.points_from_xy(odk_non_crop_df.longitude, odk_non_crop_df.latitude, crs="EPSG:4326")).to_crs(epsg=32628)

odk_non_crop_gdf = odk_non_crop_gdf.drop(columns=['longitude', 'latitude'])

odk_non_crop_gdf['geometry'] = odk_non_crop_gdf.buffer(10, resolution=24)

odk_non_crop_gdf['collect'] = 'ODK_point'

odk_non_crop_gdf

,Picture-LC_Class,Picture-Other_Non_Cropland,lc_code,lc_name,geometry,collect
0,5,NaN,5,Sol nu,"POLYGON ((394713.679 1527943.790, 394713.657 1...",ODK_point
1,5,NaN,5,Sol nu,"POLYGON ((393927.658 1529546.910, 393927.637 1...",ODK_point
2,5,NaN,5,Sol nu,"POLYGON ((394289.803 1529607.426, 394289.781 1...",ODK_point
3,5,NaN,5,Sol nu,"POLYGON ((440157.486 1521520.015, 440157.465 1...",ODK_point
4,5,NaN,5,Sol nu,"POLYGON ((451198.027 1521623.032, 451198.005 1...",ODK_point
...,...,...,...,...,...,...
75,2,NaN,2,Jachère,"POLYGON ((417104.322 1519563.856, 417104.300 1...",ODK_point
76,2,NaN,2,Jachère,"POLYGON ((1880848.806 5798867.986, 1880848.785...",ODK_point
77,2,NaN,2,Jachère,"POLYGON ((1880846.869 5798864.264, 1880846.848...",ODK_point
78,2,NaN,2,Jachère,"POLYGON ((346016.171 1774276.681, 346016.149 1...",ODK_point


# Merge ODK data with ODK geometries

In [13]:
odk_with_data_gdf = odk_gdf.merge(odk_crop_df, on='id')

print(f'There are {len(odk_with_data_gdf)} polygons obtained with the tablet (ODK)')

odk_with_data_gdf.head()

There are 215 polygons obtained with the tablet (ODK)


,id,geometry,area,collect,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name
0,6220201174341,"POLYGON ((393917.866 1528882.489, 393901.785 1...",24112.55,ODK_polygon,1,1.0,31297.99,1,Arachide
1,6220201173222,"POLYGON ((394098.110 1528308.033, 394129.216 1...",4709.83,ODK_polygon,1,13.0,9618.22,13,Pastèque
2,6220201171611,"POLYGON ((394069.867 1529538.943, 394187.774 1...",12635.46,ODK_polygon,1,1.0,17649.58,1,Arachide
3,6220201171631,"POLYGON ((393943.188 1529558.847, 393938.597 1...",6242.19,ODK_polygon,1,1.0,10117.53,1,Arachide
4,6220201171661,"POLYGON ((393939.067 1529497.632, 393938.507 1...",3883.07,ODK_polygon,1,1.0,7216.51,1,Arachide


# Merge ODK data with GPS geometries

In [24]:
gps_with_data_gdf = gps_gdf.merge(odk_crop_df, on='id', how='left')

gps_with_data_gdf['gid'] = np.arange(gps_with_data_gdf.shape[0])

print(f'There are {len(gps_with_data_gdf)} polygons obtained with GPS')

gps_with_data_gdf.head()

There are 321 polygons obtained with GPS


,filename,id,geometry,area,collect,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name,gid
0,Piste_6220102-2-4-1-1.gpx,62201022411,"POLYGON ((429457.856 1516229.002, 429465.169 1...",5172.41,GPS,1.0,1.0,5266.50,1.0,Arachide,0
1,Piste_6220102-2-4-1-2.gpx,62201022412,"POLYGON ((429828.180 1516150.340, 429839.170 1...",12162.62,GPS,1.0,11.0,16867.89,11.0,Mil,1
2,Piste_6220102-2-4-1-3.gpx,62201022413,"POLYGON ((429883.899 1516457.550, 429884.205 1...",6209.42,GPS,1.0,11.0,6282.66,11.0,Mil,2
3,Piste_6220102-2-4-1-4.gpx,62201022414,"POLYGON ((429986.736 1516281.999, 429987.433 1...",2772.11,GPS,1.0,9.0,3493.89,9.0,Maïs,3
4,Piste_6220102-2-16-4.gpx,62201022164,"POLYGON ((429722.291 1515343.246, 429740.288 1...",1860.82,GPS,NaN,NaN,NaN,NaN,NaN,4


# Concatenate ODK polygons / ODK points / GPS

In [33]:
gdf_list = [odk_with_data_gdf, odk_non_crop_gdf, gps_with_data_gdf]

full_polygons_gdf = pd.concat(gdf_list, axis=0, ignore_index=True)

full_polygons_gdf['gid'] = np.arange(full_polygons_gdf.shape[0])

full_polygons_gdf = full_polygons_gdf.to_crs(epsg=32628)

full_polygons_gdf.head()


,id,geometry,area,collect,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name,Picture-LC_Class,Picture-Other_Non_Cropland,lc_code,lc_name,filename,gid
0,6220201174341,"POLYGON ((393929.276 1528886.677, 393909.441 1...",31096.39,ODK_polygon,1.0,1.0,31297.99,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,0
1,6220201173222,"POLYGON ((394090.930 1528320.745, 394142.002 1...",9556.25,ODK_polygon,1.0,13.0,9618.22,13.0,Pastèque,NaN,NaN,NaN,NaN,NaN,1
2,6220201171611,"POLYGON ((394059.235 1529548.963, 394198.230 1...",17535.89,ODK_polygon,1.0,1.0,17649.58,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,2
3,6220201171631,"POLYGON ((393936.493 1529548.371, 393926.686 1...",10052.36,ODK_polygon,1.0,1.0,10117.53,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,3
4,6220201171661,"POLYGON ((393929.236 1529486.680, 393928.363 1...",7170.03,ODK_polygon,1.0,1.0,7216.51,1.0,Arachide,NaN,NaN,NaN,NaN,NaN,4


# Save to shapefile

In [25]:
odk_with_data_gdf.to_file(filename=odk_crop_shp_output)

odk_non_crop_gdf.to_file(filename=odk_non_crop_shp_output)

gps_with_data_gdf.to_file(filename=gps_shp_output,)

#full_polygons_gdf.to_file(filename=full_shp_output, driver='ESRI Shapefile')

# Interactive plot with `folium`

https://leafletjs.com/reference-1.6.0.html#path-option

https://python-visualization.github.io/folium/quickstart.html

https://geopandas.org/gallery/polygon_plotting_with_folium.html

https://bikeshbade.com.np/tutorials/Detail/?title=Beginner+guide+to+python+Folium+module+to+integrate+google+earth+engine&code=8

https://nbviewer.org/github/python-visualization/folium/blob/master/examples/ImageOverlay.ipynb

In [23]:
#f = folium.Figure(width=1000, height=700)

m = folium.Map(location = [13.743747099563299, -15.772308355932424], zoom_start=11.5)#.add_to(f)

basemap_dict = folium_def.get_basemap()

basemap_dict['Google Satellite'].add_to(m)

# Plot ROI

#roi_gdf = gpd.read_file(roi_shp)

#sim_geo = gpd.GeoSeries(roi_gdf['geometry']).simplify(tolerance=0.001)
#geo_j = sim_geo.to_json()
#geo_j = folium.GeoJson(data=geo_j,
#                       style_function=lambda x: {'fillOpacity': 0, 'color': 'black'})
#geo_j.add_to(m)

# Plot GPX polygons in red

for _, r in gps_with_data_gdf.to_crs(epsg=4326).iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'red'})
    
    html = f'''<b>GPS</b><br>
    ID : {r['id']}<br>
    Crop type : {r['crop_code']} - {r['crop_name']}<br>
    Area : {r['area']}
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    #folium.Popup('crop type ' + str(r['Informations_champ-Culture'])).add_to(geo_j)
    geo_j.add_to(m)

# Plot ODK cropland polygons
# --------------------------

for _, r in odk_with_data_gdf.to_crs(epsg=4326).iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'magenta'})

    html = f'''<b>ODK - Cropland</b><br>
    ID : {r['id']}<br>
    Crop type : {r['crop_code']} - {r['crop_name']}<br>
    Area : {r['area']}<br>
    Surface ODK : {r['Informations_champ-Surface']}
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    geo_j.add_to(m)

# Plot ODK non-cropland points
# ----------------------------

for _, r in odk_non_crop_gdf.to_crs(epsg=4326).iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'blue'})

    html = f'''<b>ODK - Non cropland</b><br>
    Land Cover : {r['lc_code']} - {r['lc_name']}<br>
    Comment : {r['Picture-Other_Non_Cropland']}<br>
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    geo_j.add_to(m)



m = folium_def.add_categorical_legend(m, 'Legend',
                             colors = ['red','magenta','blue'],
                             labels = ['GPX', 'ODK - cropland','ODK - non cropland'])


m.save(map_html_output)

m

KeyError: 'area'